Wczytanie danych z MovieLens

In [1]:
from surprise import Dataset
from surprise import Dataset
from surprise.model_selection import train_test_split
import pandas as pd

# Wczytaj dane MovieLens 100k
data = Dataset.load_builtin('ml-100k')


In [2]:

# Pobierz pełne dane jako listę (userId, movieId, rating)
raw_ratings = data.raw_ratings

# Przekształcenie na DataFrame
ratings_df = pd.DataFrame(raw_ratings, columns=['userId', 'movieId', 'rating', 'timestamp'])

# Wyświetl fragment danych
print(ratings_df.head())


  userId movieId  rating  timestamp
0    196     242     3.0  881250949
1    186     302     3.0  891717742
2     22     377     1.0  878887116
3    244      51     2.0  880606923
4    166     346     1.0  886397596


Przygotowanie danych binarnych

In [3]:
def to_binary(df, threshold=0):
    df['rating'] = (df['rating'] > threshold).astype(int)
    return df

binary_ratings_df = to_binary(ratings_df)

print(binary_ratings_df.head())


  userId movieId  rating  timestamp
0    196     242       1  881250949
1    186     302       1  891717742
2     22     377       1  878887116
3    244      51       1  880606923
4    166     346       1  886397596


Przekształcenie na macierz użytkownik-film

In [4]:
# Tworzenie macierzy użytkownik-film
user_movie_matrix = binary_ratings_df.pivot(index='userId', columns='movieId', values='rating').fillna(0).astype(int)

# Wyświetlenie fragmentu macierzy
print(user_movie_matrix.head())


movieId  1  10  100  1000  1001  1002  1003  1004  1005  1006  ...  990  991  \
userId                                                         ...             
1        1   1    1     0     0     0     0     0     0     0  ...    0    0   
10       1   0    1     0     0     0     0     0     0     0  ...    0    0   
100      0   0    0     0     0     0     0     0     0     0  ...    1    0   
101      1   0    0     0     0     0     0     0     0     0  ...    0    0   
102      1   0    0     0     0     0     0     0     0     0  ...    0    0   

movieId  992  993  994  995  996  997  998  999  
userId                                           
1          0    0    0    0    0    0    0    0  
10         0    0    0    0    0    0    0    0  
100        0    0    0    0    0    0    0    0  
101        0    0    0    0    0    0    0    0  
102        0    1    0    0    0    0    0    0  

[5 rows x 1682 columns]


RBM

In [5]:
from sklearn.neural_network import BernoulliRBM
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_matrix = scaler.fit_transform(user_movie_matrix)

# Inicjalizacja RBM
rbm = BernoulliRBM(n_components=100, learning_rate=0.01, n_iter=10, random_state=42)

# Trenowanie RBM
rbm.fit(scaled_matrix)

print("Model RBM został wytrenowany!")


Model RBM został wytrenowany!


In [6]:
# Wyświetlenie wag po treningu
print("Wagi po treningu (fragment):")
print(rbm.components_[:5])  

user_index = 0
user_data = scaled_matrix[user_index]  
hidden_activations = rbm.transform([user_data]) 

print("\nAktywacje warstwy ukrytej dla użytkownika 0:")
print(hidden_activations)

visible_neurons = scaled_matrix.shape[1]
print(f"\nLiczba neuronów w warstwie widocznej (liczba filmów): {visible_neurons}")
print(f"Liczba neuronów w warstwie ukrytej: {rbm.components_.shape[0]}")


Wagi po treningu (fragment):
[[ 0.05808808  0.00777018  0.04714251 ... -0.04575854 -0.03297008
  -0.04556179]
 [ 0.00137272 -0.03741144 -0.05654602 ... -0.06888501 -0.06443197
  -0.05875272]
 [-0.0252592   0.01668155  0.02713275 ... -0.05876421 -0.0488179
  -0.03784065]
 [-0.01874418 -0.04943899 -0.04475827 ... -0.04185798 -0.06026539
  -0.04753255]
 [ 0.0075708   0.00447497  0.10041393 ... -0.07295412 -0.06646638
  -0.0506779 ]]

Aktywacje warstwy ukrytej dla użytkownika 0:
[[9.99820191e-01 5.97566056e-02 3.38986945e-05 1.28060124e-02
  1.78839142e-02 3.99482870e-09 1.19941680e-05 2.12418095e-02
  9.98187545e-01 9.95141140e-01 9.92610564e-07 1.12364584e-08
  9.99748063e-01 9.98973782e-01 1.76668428e-02 9.99122605e-01
  9.99557271e-01 1.13428810e-02 3.00420629e-03 9.99632099e-01
  3.27065379e-09 9.49183017e-01 1.59205403e-04 9.98220092e-01
  2.03768077e-02 5.34863181e-09 3.67759881e-05 4.54283494e-04
  1.52606042e-02 1.36598960e-01 1.40161873e-02 1.42005692e-02
  9.99589067e-01 9.37203

In [7]:
# Wyświetlenie wag po treningu
print("Wagi po treningu (fragment):")
print(rbm.components_[:5]) 

# Dane wejściowe dla wybranego użytkownika 
user_index = 0
user_data = scaled_matrix[user_index]  
print("\nDane wejściowe (warstwa widoczna) dla użytkownika 0:")
print(user_data)

# Wyjście z warstwy ukrytej 
hidden_activations = rbm.transform([user_data]) 
print("\nAktywacje warstwy ukrytej dla użytkownika 0:")
print(hidden_activations)


# Liczba neuronów w warstwach
visible_neurons = scaled_matrix.shape[1] 
hidden_neurons = rbm.components_.shape[0]  
print(f"\nLiczba neuronów w warstwie widocznej (liczba filmów): {visible_neurons}")
print(f"Liczba neuronów w warstwie ukrytej: {hidden_neurons}")


Wagi po treningu (fragment):
[[ 0.05808808  0.00777018  0.04714251 ... -0.04575854 -0.03297008
  -0.04556179]
 [ 0.00137272 -0.03741144 -0.05654602 ... -0.06888501 -0.06443197
  -0.05875272]
 [-0.0252592   0.01668155  0.02713275 ... -0.05876421 -0.0488179
  -0.03784065]
 [-0.01874418 -0.04943899 -0.04475827 ... -0.04185798 -0.06026539
  -0.04753255]
 [ 0.0075708   0.00447497  0.10041393 ... -0.07295412 -0.06646638
  -0.0506779 ]]

Dane wejściowe (warstwa widoczna) dla użytkownika 0:
[1. 1. 1. ... 0. 0. 0.]

Aktywacje warstwy ukrytej dla użytkownika 0:
[[9.99820191e-01 5.97566056e-02 3.38986945e-05 1.28060124e-02
  1.78839142e-02 3.99482870e-09 1.19941680e-05 2.12418095e-02
  9.98187545e-01 9.95141140e-01 9.92610564e-07 1.12364584e-08
  9.99748063e-01 9.98973782e-01 1.76668428e-02 9.99122605e-01
  9.99557271e-01 1.13428810e-02 3.00420629e-03 9.99632099e-01
  3.27065379e-09 9.49183017e-01 1.59205403e-04 9.98220092e-01
  2.03768077e-02 5.34863181e-09 3.67759881e-05 4.54283494e-04
  1.5260

In [8]:
import numpy as np

def reconstruct_visible(hidden_activations, rbm):
    weights = rbm.components_  # Wagi z warstwy ukrytej 
    # Bias warstwy widocznej
    visible_bias = rbm.intercept_visible_
    # Rekonstrukcja
    visible_reconstruction = np.dot(hidden_activations, weights) + visible_bias
    # Zastosowanie sigmoidy
    visible_reconstruction = 1 / (1 + np.exp(-visible_reconstruction))
    return visible_reconstruction

# Rekonstrukcja dla użytkownika 0
visible_reconstruction = reconstruct_visible(hidden_activations, rbm)

print("\nRekonstrukcja danych dla użytkownika 0:")
print(visible_reconstruction)



Rekonstrukcja danych dla użytkownika 0:
[[0.5572118  0.20711901 0.47729783 ... 0.04908461 0.05483823 0.04410401]]


Generowanie rekomendacji

In [9]:
# Dane wejściowe dla wybranego użytkownika
user_index = 1
user_data = scaled_matrix[user_index] 

# aktywacje neuronów ukrytych
hidden_activations = rbm.transform([user_data])
print(hidden_activations.shape)
# prawdopodobieństwa filmów
visible_reconstruction = np.dot(hidden_activations, rbm.components_) + rbm.intercept_visible_
visible_reconstruction = 1 / (1 + np.exp(-visible_reconstruction))  
print(visible_reconstruction.shape)

# Sortowanie filmów według prawdopodobieństwa
recommended_movies = np.argsort(visible_reconstruction[0])[::-1]
print(recommended_movies.shape)
# Wyświetlenie 10 najlepszych rekomendacji
print("Rekomendacje filmów dla użytkownika:")
for i, movie_id in enumerate(recommended_movies[:10]):
    print(f"Film {movie_id}: Prawdopodobieństwo = {visible_reconstruction[0][movie_id]:.2%}")


(1, 100)
(1, 1682)
(1682,)
Rekomendacje filmów dla użytkownika:
Film 1128: Prawdopodobieństwo = 100.00%
Film 766: Prawdopodobieństwo = 92.32%
Film 1660: Prawdopodobieństwo = 78.71%
Film 756: Prawdopodobieństwo = 77.70%
Film 776: Prawdopodobieństwo = 77.21%
Film 764: Prawdopodobieństwo = 74.88%
Film 800: Prawdopodobieństwo = 73.75%
Film 1194: Prawdopodobieństwo = 72.62%
Film 1043: Prawdopodobieństwo = 70.76%
Film 1338: Prawdopodobieństwo = 70.11%


Drugie podejście czyli każdy film ma 5 wejsć z oceanmi- 1,2,3,4,5->[0,0,0,1,0]

In [16]:
import numpy as np
import pandas as pd

def prepare_extended_matrix(df, num_movies, max_rating=5):
    
    num_users = df['userId'].nunique()
    
    # (liczba użytkowników × (liczba filmów * max_rating))
    extended_matrix = np.zeros((num_users, num_movies * max_rating), dtype=int)
    
    grouped = df.groupby('userId')
    
    for user_idx, (user_id, group) in enumerate(grouped):
        for _, row in group.iterrows():
            movie_id = int(row['movieId']) - 1 
            rating = int(row['rating'])
            if rating > 0:
                col_index = movie_id * max_rating + (rating - 1)
                extended_matrix[user_idx, col_index] = 1
    
    return extended_matrix


num_movies = ratings_df['movieId'].nunique()

extended_matrix = prepare_extended_matrix(ratings_df, num_movies)

print(f"Rozmiar macierzy rozszerzonej: {extended_matrix.shape}")

print("\nPrzykładowy wektor dla użytkownika 0 (pierwsze 50 elementów):")
print(extended_matrix[0, :50])


Rozmiar macierzy rozszerzonej: (943, 8410)

Przykładowy wektor dla użytkownika 0 (pierwsze 50 elementów):
[1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0
 0 0 0 1 0 0 0 0 1 0 0 0 0]


In [18]:
# Sprawdzenie wymiarów macierzy
num_users, num_columns = extended_matrix.shape
expected_columns = num_movies * 5

print(f"Liczba użytkowników: {num_users}")
print(f"Liczba kolumn (filmy × oceny): {num_columns}")
print(f"Oczekiwane kolumny (filmy × max_rating): {expected_columns}")

if num_columns == expected_columns:
    print("Wymiary macierzy są poprawne!")
else:
    print("Rozmiary macierzy są niezgodne. Sprawdź dane wejściowe.")


Liczba użytkowników: 943
Liczba kolumn (filmy × oceny): 8410
Oczekiwane kolumny (filmy × max_rating): 8410
Wymiary macierzy są poprawne!


In [20]:
from sklearn.neural_network import BernoulliRBM
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_extended_matrix = scaler.fit_transform(extended_matrix)

rbm = BernoulliRBM(n_components=100, learning_rate=0.01, n_iter=10, random_state=42)

rbm.fit(scaled_extended_matrix)

print("Model RBM został wytrenowany!")


Model RBM został wytrenowany!


In [21]:
import numpy as np

def reconstruct_visible(user_data, rbm):
    hidden_activations = rbm.transform([user_data])  
    visible_reconstruction = np.dot(hidden_activations, rbm.components_) + rbm.intercept_visible_
    return 1 / (1 + np.exp(-visible_reconstruction)) 

# Rekonstrukcja danych dla użytkownika 0
user_index = 0
user_data = scaled_extended_matrix[user_index]

reconstructed_visible = reconstruct_visible(user_data, rbm)

print("\nRekonstrukcja danych dla użytkownika 0 (pierwsze 50 elementów):")
print(reconstructed_visible[0][:50])



Rekonstrukcja danych dla użytkownika 0 (pierwsze 50 elementów):
[1.         0.0027414  0.00249866 0.00281948 0.00241648 0.17399763
 0.00175679 0.00234223 0.00140121 0.00131087 0.26041403 0.00183524
 0.00174141 0.00248961 0.00197546 0.29219776 0.00179024 0.00158677
 0.00327751 0.00265992 0.17583817 0.00193314 0.00281181 0.0028328
 0.00197356 0.10055599 0.00148281 0.00211982 0.00185716 0.00173331
 1.         0.00309916 0.00177511 0.00236778 0.00255543 0.41938644
 0.0015505  0.00176471 0.00311302 0.00180756 1.         0.00257108
 0.00297273 0.00118519 0.00183274 0.1398384  0.0025235  0.00122678
 0.00236819 0.00207906]


In [30]:
def get_recommendations(reconstructed, num_movies, max_rating=5):
    recommendations = []
    for movie_id in range(num_movies):
        start_idx = movie_id * max_rating
        end_idx = start_idx + max_rating
        probabilities = reconstructed[0][start_idx:end_idx]
        best_rating = np.argmax(probabilities) + 1
        best_prob = probabilities[best_rating - 1]
        recommendations.append((movie_id + 1, best_rating, best_prob))
    return sorted(recommendations, key=lambda x: x[2], reverse=True)

recommendations = get_recommendations(reconstructed_visible, num_movies)

# Wyświetlenie top 10 rekomendacji
print("\nTop 10 rekomendacji dla użytkownika 0:")
for i, (movie_id, rating, prob) in enumerate(recommendations[:10]):
    print(f"Film {movie_id}: Ocena = {rating}, Prawdopodobieństwo = {prob:.2%}")



Top 10 rekomendacji dla użytkownika 0:
Film 1: Ocena = 1, Prawdopodobieństwo = 100.00%
Film 7: Ocena = 1, Prawdopodobieństwo = 100.00%
Film 9: Ocena = 1, Prawdopodobieństwo = 100.00%
Film 22: Ocena = 1, Prawdopodobieństwo = 100.00%
Film 50: Ocena = 1, Prawdopodobieństwo = 100.00%
Film 56: Ocena = 1, Prawdopodobieństwo = 100.00%
Film 69: Ocena = 1, Prawdopodobieństwo = 100.00%
Film 79: Ocena = 1, Prawdopodobieństwo = 100.00%
Film 98: Ocena = 1, Prawdopodobieństwo = 100.00%
Film 100: Ocena = 1, Prawdopodobieństwo = 100.00%


In [27]:
# Sprawdzenie danych wejściowych do RBM
print("\nFragment danych wejściowych do RBM (pierwsze 10 wierszy):")
print(scaled_extended_matrix[:10, :10])



Fragment danych wejściowych do RBM (pierwsze 10 wierszy):
[[1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [28]:
# Sprawdzenie aktywacji warstwy ukrytej
hidden_activations = rbm.transform([user_data])
print("\nAktywacje warstwy ukrytej (pierwsze 10 wartości):")
print(hidden_activations[0][:10])



Aktywacje warstwy ukrytej (pierwsze 10 wartości):
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [29]:
# Sprawdzenie rekonstrukcji dla pierwszego użytkownika (pierwsze 50 wartości)
print("\nFragment zrekonstruowanej warstwy widocznej (pierwsze 50 wartości):")
print(reconstructed_visible[0][:50])



Fragment zrekonstruowanej warstwy widocznej (pierwsze 50 wartości):
[1.         0.0027414  0.00249866 0.00281948 0.00241648 0.17399763
 0.00175679 0.00234223 0.00140121 0.00131087 0.26041403 0.00183524
 0.00174141 0.00248961 0.00197546 0.29219776 0.00179024 0.00158677
 0.00327751 0.00265992 0.17583817 0.00193314 0.00281181 0.0028328
 0.00197356 0.10055599 0.00148281 0.00211982 0.00185716 0.00173331
 1.         0.00309916 0.00177511 0.00236778 0.00255543 0.41938644
 0.0015505  0.00176471 0.00311302 0.00180756 1.         0.00257108
 0.00297273 0.00118519 0.00183274 0.1398384  0.0025235  0.00122678
 0.00236819 0.00207906]


In [34]:
import numpy as np
import pandas as pd

def prepare_extended_matrix(df, num_movies, max_rating=5):
    num_users = df['userId'].nunique()
    
    # liczba użytkowników × (liczba filmów * max_rating)
    extended_matrix = np.zeros((num_users, num_movies * max_rating), dtype=int)

    user_map = {user_id: idx for idx, user_id in enumerate(df['userId'].unique())}
    
    for _, row in df.iterrows():
        user_idx = user_map[row['userId']]
        movie_id = int(row['movieId']) - 1 
        rating = int(row['rating'])
        if rating > 0:
            col_index = movie_id * max_rating + (rating - 1)
            extended_matrix[user_idx, col_index] = 1
    
    return extended_matrix

num_movies = ratings_df['movieId'].nunique()
extended_matrix = prepare_extended_matrix(ratings_df, num_movies)

print(f"Rozmiar macierzy rozszerzonej: {extended_matrix.shape}")

# Wyświetlenie przykładowego wiersza (dla użytkownika 0)
print("\nPrzykładowy wiersz dla użytkownika 0 (pierwsze 50 elementów):")
print(extended_matrix[0, :50])


Rozmiar macierzy rozszerzonej: (943, 8410)

Przykładowy wiersz dla użytkownika 0 (pierwsze 50 elementów):
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [40]:
import numpy as np
from sklearn.neural_network import BernoulliRBM
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_extended_matrix = scaler.fit_transform(extended_matrix)

rbm = BernoulliRBM(n_components=200, learning_rate=0.005, n_iter=20, random_state=42)

rbm.fit(scaled_extended_matrix)
print("Model RBM został wytrenowany!")

print("\nFragment wag RBM po treningu (pierwsze 5 komponentów):")
print(rbm.components_[:5])

# Rekonstrukcja warstwy widocznej dla użytkownika 0
def reconstruct_visible(user_data, rbm):
    hidden_activations = rbm.transform([user_data])  
    visible_reconstruction = np.dot(hidden_activations, rbm.components_) + rbm.intercept_visible_
    return 1 / (1 + np.exp(-visible_reconstruction)) 

user_index = 0
user_data = scaled_extended_matrix[user_index]
reconstructed_visible = reconstruct_visible(user_data, rbm)

print("\nRekonstrukcja danych dla użytkownika 0 (pierwsze 50 elementów):")
print(reconstructed_visible[0][:50])

# Funkcja do wyciągania rekomendacji
def get_recommendations(reconstructed, num_movies, max_rating=5):
    recommendations = []
    for movie_id in range(num_movies):
        start_idx = movie_id * max_rating
        end_idx = start_idx + max_rating
        probabilities = reconstructed[0][start_idx:end_idx]
        best_rating = np.argmax(probabilities) + 1
        best_prob = probabilities[best_rating - 1]
        recommendations.append((movie_id + 1, best_rating, best_prob))
    return sorted(recommendations, key=lambda x: x[2], reverse=True)

recommendations = get_recommendations(reconstructed_visible, num_movies)

# Wyświetlenie top 10 rekomendacji
print("\nTop 10 rekomendacji dla użytkownika 0:")
for i, (movie_id, rating, prob) in enumerate(recommendations[:200]):
    print(f"Film {movie_id}: Ocena = {rating}, Prawdopodobieństwo = {prob:.2%}")


Model RBM został wytrenowany!

Fragment wag RBM po treningu (pierwsze 5 komponentów):
[[ 1.37541853 -0.02833744 -0.02258754 ... -0.04431785 -0.03174618
  -0.02567736]
 [ 1.36585201 -0.01664808 -0.0394924  ... -0.01190574 -0.04480319
  -0.01864273]
 [ 1.35797884 -0.03631216 -0.04906067 ... -0.02994848 -0.03123301
  -0.04381288]
 [ 1.37747148 -0.02667361 -0.03610937 ... -0.03437899 -0.01178573
  -0.01147151]
 [ 1.37470415 -0.03542508 -0.0388136  ... -0.02750288 -0.02112261
  -0.01714202]]

Rekonstrukcja danych dla użytkownika 0 (pierwsze 50 elementów):
[1.00000000e+00 7.47064552e-04 4.91871453e-04 1.14556198e-03
 5.78597298e-04 1.65866886e-01 7.93756981e-04 1.13364444e-03
 8.53037187e-04 1.08550808e-03 1.50509923e-01 9.14501403e-04
 7.94466355e-04 1.09406438e-03 9.57137161e-04 2.30647974e-01
 1.86127314e-03 1.30036149e-03 1.46110907e-03 1.26263109e-03
 1.47430230e-01 1.16146505e-03 8.83584150e-04 1.07011071e-03
 1.49659274e-03 4.29362287e-02 1.17784963e-03 1.12768724e-03
 7.82513289e-04 

In [42]:
import numpy as np
import pandas as pd
from sklearn.neural_network import BernoulliRBM
from sklearn.preprocessing import MinMaxScaler
from surprise import Dataset

# Wczytanie danych MovieLens z surprise
data = Dataset.load_builtin('ml-100k')
raw_ratings = data.raw_ratings  

ratings_df = pd.DataFrame(raw_ratings, columns=['userId', 'movieId', 'rating', 'timestamp'])

# Przygotowanie danych wejściowych
def prepare_extended_matrix(df, num_movies, max_rating=5):
    num_users = df['userId'].nunique()
    extended_matrix = np.zeros((num_users, num_movies * max_rating), dtype=int)
    user_map = {user_id: idx for idx, user_id in enumerate(df['userId'].unique())}
    
    for _, row in df.iterrows():
        user_idx = user_map[row['userId']]
        movie_id = int(row['movieId']) - 1  
        rating = int(row['rating'])
        if rating > 0:
            col_index = movie_id * max_rating + (rating - 1)
            extended_matrix[user_idx, col_index] = 1
    
    return extended_matrix

# Liczba unikalnych filmów
num_movies = ratings_df['movieId'].nunique()

# Przygotowanie macierzy rozszerzonej
extended_matrix = prepare_extended_matrix(ratings_df, num_movies)

# Sprawdzenie rozmiaru macierzy
print(f"Rozmiar macierzy rozszerzonej: {extended_matrix.shape}")

# Skalowanie danych do zakresu [0, 1]
scaler = MinMaxScaler()
scaled_extended_matrix = scaler.fit_transform(extended_matrix)

# Inicjalizacja RBM
rbm = BernoulliRBM(n_components=500, learning_rate=0.005, n_iter=20, random_state=42)

# Trenowanie RBM
rbm.fit(scaled_extended_matrix)
print("Model RBM został wytrenowany!")

# Rekonstrukcja warstwy widocznej
def reconstruct_visible(user_data, rbm):
    hidden_activations = rbm.transform([user_data])  
    visible_reconstruction = np.dot(hidden_activations, rbm.components_) + rbm.intercept_visible_
    return 1 / (1 + np.exp(-visible_reconstruction)) 

# Rekonstrukcja danych dla użytkownika 0
user_index = 0
user_data = scaled_extended_matrix[user_index]
reconstructed_visible = reconstruct_visible(user_data, rbm)

# Wyświetlenie rekonstrukcji dla pierwszych 10 filmów
print("\nRekonstrukcja dla pierwszych 10 filmów:")
for i in range(10):
    start_idx = i * 5
    end_idx = start_idx + 5
    print(f"Film {i + 1}: {reconstructed_visible[0][start_idx:end_idx]}")

# Funkcja generowania rekomendacji z samplingiem
def get_recommendations_with_sampling(reconstructed, num_movies, max_rating=5):
    recommendations = []
    for movie_id in range(num_movies):
        start_idx = movie_id * max_rating
        end_idx = start_idx + max_rating
        probabilities = reconstructed[0][start_idx:end_idx]
        probabilities /= probabilities.sum()  
        sampled_rating = np.random.choice(range(1, max_rating + 1), p=probabilities)
        recommendations.append((movie_id + 1, sampled_rating, probabilities[sampled_rating - 1]))
    return sorted(recommendations, key=lambda x: x[2], reverse=True)

# Generowanie rekomendacji
recommendations = get_recommendations_with_sampling(reconstructed_visible, num_movies)

# Wyświetlenie top 10 rekomendacji
print("\nTop 10 rekomendacji dla użytkownika 0:")
for i, (movie_id, rating, prob) in enumerate(recommendations[:10]):
    print(f"Film {movie_id}: Ocena = {rating}, Prawdopodobieństwo = {prob:.2%}")


Rozmiar macierzy rozszerzonej: (943, 8410)
Model RBM został wytrenowany!

Rekonstrukcja dla pierwszych 10 filmów:
Film 1: [0.0063335  0.02901742 0.28485954 0.11460529 0.03855323]
Film 2: [0.01603595 0.00710742 0.04935667 0.02222455 0.0187859 ]
Film 3: [0.00948203 0.01643805 0.01137839 0.02772219 0.01633022]
Film 4: [0.00382283 0.02063557 0.06511463 0.03878246 0.01476369]
Film 5: [0.00320774 0.00505579 0.01873569 0.07566866 0.00641733]
Film 6: [0.00166979 0.00450765 0.0050816  0.00726675 0.01196514]
Film 7: [0.00767102 0.02035012 0.05824723 0.03161896 0.02848914]
Film 8: [0.00461933 0.02963954 0.16563076 0.02485076 0.04875979]
Film 9: [0.01555731 0.03486274 0.02424723 0.04523946 0.19637373]
Film 10: [0.00244963 0.00384216 0.02974657 0.03329223 0.02161951]

Top 10 rekomendacji dla użytkownika 0:
Film 1330: Ocena = 3, Prawdopodobieństwo = 91.80%
Film 439: Ocena = 1, Prawdopodobieństwo = 90.20%
Film 247: Ocena = 1, Prawdopodobieństwo = 88.25%
Film 1538: Ocena = 3, Prawdopodobieństwo = 88.2

In [46]:
import matplotlib.pyplot as plt

# 1. Liczba filmów i ocen na użytkownika
user_nonzero = np.count_nonzero(extended_matrix, axis=1)
print(f"Średnia liczba ocen na użytkownika: {np.mean(user_nonzero):.2f}")
print(f"Minimalna liczba ocen na użytkownika: {np.min(user_nonzero)}")
print(f"Maksymalna liczba ocen na użytkownika: {np.max(user_nonzero)}")


# 2. Rozkład wartości w macierzy wejściowej
nonzero_values = np.count_nonzero(extended_matrix)
total_values = extended_matrix.size
sparsity = 100 * (1 - nonzero_values / total_values)

print(f"Liczba niezerowych wartości: {nonzero_values}")
print(f"Całkowita liczba wartości w macierzy: {total_values}")

# 3. Fragment macierzy dla użytkownika 0
print("\nFragment macierzy dla użytkownika 0 (pierwsze 50 kolumn):")
print(extended_matrix[0, :50])

# 4. Długość wektora użytkownika
print(f"\nDługość wektora użytkownika 0: {len(extended_matrix[0])}")


Średnia liczba ocen na użytkownika: 106.04
Minimalna liczba ocen na użytkownika: 20
Maksymalna liczba ocen na użytkownika: 737
Liczba niezerowych wartości: 100000
Całkowita liczba wartości w macierzy: 7930630

Fragment macierzy dla użytkownika 0 (pierwsze 50 kolumn):
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0]

Długość wektora użytkownika 0: 8410


In [56]:
user_index = 100
user_data = scaled_extended_matrix[user_index]

reconstructed_visible = reconstruct_visible(user_data, rbm)

print(f"\nRekonstrukcja dla użytkownika {user_index} (pierwszych 10 filmów):")
for i in range(10):
    start_idx = i * 5
    end_idx = start_idx + 5
    print(f"Film {i + 1}: {reconstructed_visible[0][start_idx:end_idx]}")

recommendations = get_recommendations_with_sampling(reconstructed_visible, num_movies)

# Wyświetlenie top 10 rekomendacji dla wybranego użytkownika
print(f"\nTop 10 rekomendacji dla użytkownika {user_index}:")
for i, (movie_id, rating, prob) in enumerate(recommendations[:10]):
    print(f"Film {movie_id}: Ocena = {rating}, Prawdopodobieństwo = {prob:.2%}")



Rekonstrukcja dla użytkownika 100 (pierwszych 10 filmów):
Film 1: [0.0067519  0.03045835 0.28526261 0.11706319 0.03996236]
Film 2: [0.01681953 0.00763884 0.05169699 0.02332135 0.01952578]
Film 3: [0.01002911 0.01729319 0.01211262 0.02856454 0.0170686 ]
Film 4: [0.00412972 0.02186492 0.06756628 0.04135187 0.01572173]
Film 5: [0.00346371 0.00547933 0.01973245 0.07820662 0.00684161]
Film 6: [0.00180961 0.00482305 0.00541336 0.00775381 0.01246135]
Film 7: [0.00810674 0.02140985 0.06012068 0.03311185 0.02986455]
Film 8: [0.00492189 0.03048356 0.16871551 0.02617055 0.0506625 ]
Film 9: [0.01623647 0.03580852 0.02566579 0.0469205  0.19845182]
Film 10: [0.00264725 0.00413273 0.03076677 0.03467169 0.02230334]

Top 10 rekomendacji dla użytkownika 100:
Film 1330: Ocena = 3, Prawdopodobieństwo = 91.33%
Film 439: Ocena = 1, Prawdopodobieństwo = 89.59%
Film 1538: Ocena = 3, Prawdopodobieństwo = 87.64%
Film 247: Ocena = 1, Prawdopodobieństwo = 87.63%
Film 1124: Ocena = 4, Prawdopodobieństwo = 86.31%
